# Рекомендации тарифов


**Описание проекта:**\
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.


**Описание данных:**\
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

`сalls` — количество звонков, \
`minutes` — суммарная длительность звонков в минутах, \
`messages` — количество sms-сообщений, \
`mb_used` — израсходованный интернет-трафик в Мб,\
`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [3]:
import pandas as pd #импортируем необходимые библиотеки
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [4]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv') 
display(df)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
df['calls'] = df['calls'].astype('Int64') #приведем звонки и сообщения к целочисленным величинам
df['messages'] = df['messages'].astype('Int64')

In [5]:
display(df)

,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0
...,...,...,...,...,...
3209,122,910.98,20,35124.90,1
3210,25,190.36,0,3275.61,0
3211,97,634.44,70,13974.06,0
3212,64,462.32,90,31239.78,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   Int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   Int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: Int64(2), float64(2), int64(1)
memory usage: 131.9 KB


## Разбейте данные на выборки

Для начала зададим признаки и целевой признак. В нашем случае целевой признак - это тариф, а параметры - звонки, минуты, сообщения и мегабайты.

In [7]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Так как сркытой тестовой выборки нет, разделим датасет на 3 выборки - обучающую (60%), валидационную(20%) и тестовую(20%). Для этого сначала выделим 20% данных для тестовой выборки, а затем разделим оставшиеся 80% на обучающую и валидационную выборки, задав размер валидациооной выборки - 25%.

In [8]:
x, features_test, y, target_test = train_test_split(features, target, test_size=0.2, random_state=12345) #отделяем 20% для тестовой выборки

In [9]:
features_train, features_valid, target_train, target_valid = train_test_split(x, y, test_size=0.25, random_state=12345) #делим оставшуюся часть на обучающую и валидационную выборки

In [10]:
print(features_train.shape) #проверим размеры выборок
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(target_test.shape)
print(features_test.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643,)
(643, 4)


## Исследуйте модели

Рассмотрим 3 модели машинного обучения - решающее дерево, рандомный лес и линейную регрессию. Для определения лучшей модели рассчитаем точность (accuracy).

Поскольку наш целевой признак является категориальным, нам нужны модели классификации.

Начнем с дерева решений. 

### Decision Tree

In [12]:
best_model_dt = None
best_result_dt = 0
for depth in range(1, 20):
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=depth, criterion='gini', min_samples_leaf=1, min_samples_split=2) # обучим модель с заданной глубиной дерева
    model_dt.fit(features_train, target_train) # обучим модель
    predictions_dt = model_dt.predict(features_valid) # получим предсказания модели
    result_dt = accuracy_score(target_valid, predictions_dt) # посчитаем качество модели
    if result_dt > best_result_dt:
        best_model_dt = model_dt    
        best_result_dt = result_dt
        
print("Точность лучшей модели дерева решений на валидационной выборке:", best_result_dt)  

Точность лучшей модели дерева решений на валидационной выборке: 0.7744945567651633


<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Рассмотрен только один гиперпараметр, да и то в котором не очень большой диапазон значений для перебора параметров. Хотелось бы побольше что-нибудь посмотреть

### Random Forest

In [13]:
best_model_rf = None
best_result_rf = 0
for est in range(1, 11):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=est, min_samples_leaf=1, min_samples_split=2) # обучим модель с заданным количеством деревьев
    model_rf.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result_rf = model_rf.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result_rf > best_result_rf:
        best_model_rf = model_rf # сохраним наилучшую модель
        best_result_rf = result_rf #  сохраним наилучшее значение метрики accuracy на валидационных данных

print("Точность наилучшей модели случайного леса на валидационной выборке:", best_result_rf)

Точность наилучшей модели случайного леса на валидационной выборке: 0.7884914463452566


<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Аналогично с прошлым комментарием

### Logistic Regression

In [126]:
model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) # обучим модель с заданным гиперпараметрами
model_lr.fit(features_train, target_train) # обучим модель на тренировочной выборке
#model_lr.predict(new_item)
predictions_lr = model_lr.predict(features_valid) # получим предсказания модели
result_lr = model_lr.score(features_valid, target_valid) # посчитаем качество модели
print("Точность логистической регрессии на валидационной выборке:", result_lr)

Точность логистической регрессии на валидационной выборке: 0.7262830482115086


<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Тут вообще не изучены гиперпараметры :(

По результатам исследования моделей можно сделать вывод, что наиболее подходящей моделью является ***случайный лес***, так как у нее наибольшее значение accuracy.

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 
    
Для более красивого оформления можно здесь использовать f-strings. Подробнее про них почитать можешь здесь или на любом другом сайте: https://python-scripts.com/f-strings

## Проверьте модель на тестовой выборке

Проверим все три модели на тестовой выборке. 

### Decision Tree

In [117]:
predictions_dt_test = model_dt.predict(features_test) # получите предсказания модели
result_dt_test = accuracy_score(target_test, predictions_dt)
print("Точность дерева решений на тестовой выборке:", result_dt_test)

Точность дерева решений на тестовой выборке: 0.6127527216174183


### Random Forest

In [118]:
result_rf_test = model_rf.score(features_test, target_test)
print("Точность случайного леса на тестовой выборке:", result_rf_test)

Точность случайного леса на тестовой выборке: 0.7869362363919129


### Logistic Regression

In [119]:
predictions_lr_test = model_lr.predict(features_test)
result_lr_test = model_lr.score(features_test, target_test)
print("Точность дерева решений на тестовой выборке:", result_lr_test)

Точность дерева решений на тестовой выборке: 0.7589424572317263


Результаты проверки на тестовой выборке также показывают наивысшую точность при применении модели *случайного леса*. В случае дерева решений точность на дереве решений ниже, а логистической регресии - выше.

Точность *случайного леса* на **валидационной выборке**: 0.7884914463452566 \
Точность *случайного леса* на **тестовой выборке**: 0.7869362363919129

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Модели проверены на тесте

## (бонус) Проверьте модели на адекватность

Проверим модель на адекватность с помощью DummyClassifier.

In [123]:
model_dc = DummyClassifier(random_state=12345, strategy='most_frequent')
model_dc.fit(features_train, target_train)
result_dc_valid = model_dc.score(features_valid, target_valid)
result_dc_test = model_dc.score(features_test, target_test)
print('Точность DummyClassifier на валидационной выборке:', result_dc_valid)
print('Точность DummyClassifier на тестовой выборке:', result_dc_test)

Точность Dummy classifier на валидационной выборке: 0.6889580093312597
Точность Dummy classifier на тестовой выборке: 0.6951788491446346


Согласно результатам проверки на адекватность, dummy-модель (предсказывает наиболее часто встречающийся класс) демонстрирует более низкую точность, чем случайный лес, что говорит о том, что модель *случайный лес* адекватна.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Можно проще, посмотреть на баланс классов: например, если у нас 90 процентов таргета это "1", а остальные "10" - 0, то мы можем тупо выдавать каждый раз цифру 1 и получать метрику 90 %. Любая модель, которая будет выдавать качество ниже этого - плохая/неадекватная

## Выводы

1. Данные предварительно были предобработаны. Значения в столбцах calls и messages были приведены к целочисленным.
2. Далее датасет был разделен на 3 выборки - обучающую (60%), валидационную (20%) и тестовую (20%).
3. Поскольку целевой признак является категориальным, были были проанализированы модели классификации: дерево решений (DecisionTreeClassifier), случайный лес (RandomForestClassifier) и  логистическая регрессия (LogisticRegression).
4. Сначала все модели были обучены на обучающей выборке и проверены на точность на валидационной выборке. Затем точность (accuracy) по валидационной выборке сравнилась с точностью на тестовой выборке.

**По результатам проекта было установлено, что проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра» с наивысшей точность (>0.75) может модель Random Forest.**

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Хороший проект, тобою проделана большая работа, ты молодец!

Сперва похвалю за наличие везде промежуточных выводов, комментариев. Хороший код, показываешь навыки владения sklearn. Все требуемые пункты в задании в целом выполнены. Оставил пару желтых комментариев, думаю, это сможет улучшить твой проект. Также можно добавить визуализацию в проект: посмотреть на графиках, как влияет перебор параметров на результат нашей метрики

В этом проекте большое внимание уделяется подбору гиперпараметров, но это можно автоматизировать! Советую почитать на досуге про GridsearchCV и Optuna, будет полезно
    
Если хочешь, отправляй проект и так, без доработок. Готов принять, потому что не было критических ошибок.

Удачи!

<div class="alert alert-info">
<font size="4", color = "black"><b>✍ Комментарий студента</b></font>
    <br /> 
    <font size="3", color = "black">
<br /> 
        Большое спасибо за правки и предложения! Про GridsearchCV и Optuna обязательно почитаю, сегодня говорили о них на вебинаре.
        Пока с гиперпараметрами непросто. Понимаю, зачем они нужны (те, что я дописала), но не очень понимаю, как определять оптимальные (кроме depth, потому что его разбирали). У нас в курсе было только про дерево решений. Поскольку лес - это много деревьев, предполагаю, его гиперпараметры применимы и к random forest, а вот про логистическую регрессию еще буду читать. В курсе дано только то, что я написала, остальное "будет рассмотрено в следующих темах".
        Честно говоря, я уже для бонусного задания дополнительно читала, так как этого тоже не было в модуле)

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

"Определять оптимальные" - вообще в целом сложно, нет какой-то идеальных значений гиперпараметра, которые принесут успех. Поэтому и занимаемся такими подборами церез циклы - но да, можно легче через GridSearchCV

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
